In [1]:
import sys, os
import multiprocessing as mp
from joblib import Parallel, delayed

import numpy as np
import matplotlib.pyplot as plt

from power_ksamp import power_ksamp_angle
from hyppo.independence import CCA, Dcorr, HHG, Hsic, RV, MGC, KMERF
from hyppo.tools import *

sys.path.append(os.path.realpath('..'))

In [2]:
import seaborn as sns
sns.set(color_codes=True, style='white', context='talk', font_scale=1.5)
PALETTE = sns.color_palette("Set1")
sns.set_palette(PALETTE[1:5] + PALETTE[6:])

These are some constants that are used in this notebook. If running these notebook, please only manipulate these constants if you are not running more tests. They define the sample sizes tested upon and the number of replications. The simulations tested over and the independence tests tested over are defined also.

In [3]:
from rpy2.robjects import Formula, numpy2ri
from rpy2.robjects.packages import importr


class Manova:
    r"""
    Wrapper of R MANOVA
    """
    def __init__(self):
        self.stats = importr('stats')
        self.r_base = importr('base')
        
        numpy2ri.activate()

        self.formula = Formula('X ~ Y')
        self.env = self.formula.environment

    def statistic(self, x, y):
        r"""
        Helper function to calculate the test statistic
        """
        self.env['Y'] = y
        self.env['X'] = x

        stat = self.r_base.summary(self.stats.manova(self.formula), test="Pillai")[3][0, 1]

        return stat

In [4]:
MAX_ANGLE = 90
STEP_SIZE = 5
ANGLES = range(1, MAX_ANGLE + STEP_SIZE, STEP_SIZE)
POWER_REPS = 5

In [5]:
SIMULATIONS = {
    "linear": "Linear",
    "exponential": "Exponential",
    "cubic": "Cubic",
    "joint_normal": "Joint Normal",
    "step": "Step",
    "quadratic": "Quadratic",
    "w_shaped": "W-Shaped",
    "spiral": "Spiral",
    "uncorrelated_bernoulli": "Bernoulli",
    "logarithmic": "Logarithmic",
    "fourth_root": "Fourth Root",
    "sin_four_pi": "Sine 4\u03C0",
    "sin_sixteen_pi": "Sine 16\u03C0",
    "square": "Square",
    "two_parabolas": "Two Parabolas",
    "circle": "Circle",
    "ellipse": "Ellipse",
    "diamond": "Diamond",
    "multiplicative_noise": "Multiplicative",
    "multimodal_independence": "Independence"
}

TESTS = [
    KMERF,
    MGC,
    Dcorr,
    Hsic,
    Manova,
    HHG,
    CCA,
    RV,
]

The following function calculates the estimated power ``POWER_REPS`` number off times and averages them. It does this iterating over the number of sample sizes.

**Note: We only recommend running this code if running the next 2 cells ONCE to generate the csv files used to visualize the plots. This code takes a very long time to run and if running, we recommend using a machine with many cores.**

In [6]:
def estimate_power(sim, test):
    est_power = np.array([np.mean([power_ksamp_angle(test, rot_ksamp, sim, angle=i) for _ in range(POWER_REPS)])
                          for i in ANGLES])
    np.savetxt('../ksample/ksamp_vs_angle/{}_{}.csv'.format(sim, test.__name__),
               est_power, delimiter=',')
    
    return est_power

In [7]:
outputs = Parallel(n_jobs=-1, verbose=100)(
    [delayed(estimate_power)(sim, test) for sim in SIMULATIONS.keys() for test in TESTS]
)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Unable to determine R library path: Command '('/Library/Frameworks/R.framework/Resources/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.
Unable to determine R library path: Command '('/Library/Frameworks/R.framework/Resources/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.


[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  2.2min


Unable to determine R library path: Command '('/Library/Frameworks/R.framework/Resources/bin/Rscript', '-e', 'cat(Sys.getenv("LD_LIBRARY_PATH"))')' returned non-zero exit status 2.


[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed:  2.6min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed:  7.9min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 22.0min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 22.1min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 25.4min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 31.1min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 31.3min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 31.4min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 34.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 35.1min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 49.3min
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed: 52.3min


...

[Parallel(n_jobs=-1)]: Done 135 out of 140 | elapsed: 1868.5min remaining: 69.2min


R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 

R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



[Parallel(n_jobs=-1)]: Done 137 out of 140 | elapsed: 2199.8min remaining: 48.2min


R[write to console]: There were 50 or more warnings (use warnings() to see the first 50)
R[write to console]: 



[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed: 2269.4min finished


The following code loops over each saved independence test file and generates absolute power curves for each test and for each simulation modality.

In [19]:
def plot_power():
    fig, ax = plt.subplots(nrows=4, ncols=5, figsize=(25,20))
    
    plt.suptitle("Multivariate Three-Sample Testing Increasing Angle", y=0.93, va='baseline')
    
    for i, row in enumerate(ax):
        for j, col in enumerate(row):
            count = 5*i + j
            sim = list(SIMULATIONS.keys())[count]
            
            for test in TESTS:
                test_name = test.__name__
                power = np.genfromtxt('../ksample/ksamp_vs_angle/{}_{}.csv'.format(sim, test_name),
                                      delimiter=',')
#                 manova_power = np.genfromtxt('../ksample/ksamp_vs_angle/{}_Manova.csv'.format(sim),
#                                           delimiter=',')
                
                if test_name == "MGC":
                    col.plot(ANGLES, power, color="#e41a1c", label=test_name, lw=4)
                elif test_name == "KMERF":
                    col.plot(ANGLES, power, color="#e41a1c", label=test_name, lw=4, linestyle='dashed')
                elif test_name == "Manova":
                    col.plot(ANGLES, power, color="#000000", label=test_name, lw=4)
                else:
                    col.plot(ANGLES, power, label=test_name, lw=2)
                col.set_xticks([])
                if i == 3:
                    col.set_xticks([ANGLES[0] - 1, ANGLES[-1] - 1])
                col.set_ylim(-0.05, 1.05)
                col.set_yticks([])
                if j == 0:
                    col.set_yticks([0, 1])
                col.set_title(SIMULATIONS[sim])
    
    fig.text(0.5, 0.08, 'Angle', ha='center')
    fig.text(0.08, 0.5, 'Statistical Power Relative to Manova', va='center', rotation='vertical')
    leg = plt.legend(bbox_to_anchor=(0.5, 0.07), bbox_transform=plt.gcf().transFigure,
                     ncol=len(TESTS), loc='upper center')
    leg.get_frame().set_linewidth(0.0)
    for legobj in leg.legendHandles:
        legobj.set_linewidth(5.0)
    plt.subplots_adjust(hspace=.50)
    plt.savefig('../ksample/figs/ksamp_power_angle.pdf', transparent=True, bbox_inches='tight')

In [20]:
plot_power()